In [1]:
%load_ext bigdata
%hive_start
%timeout 300

In [2]:
!hadoop fs -rm -f -r tmp;

Deleted tmp


In [3]:
%%hive
DROP TABLE IF EXISTS tbl0;
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';

DROP TABLE IF EXISTS tbl0;
OK
Time taken: 12.244 seconds
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 0.857 seconds


In [4]:
%%hive
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;

LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
Loading data to table default.tbl0
OK
Time taken: 1.411 seconds


In [9]:
%%hive
DROP TABLE IF EXISTS salida;
CREATE TABLE salida
AS
    SELECT t0.ano, t0.letra, count(*) AS conteo
    FROM
        (
        SELECT YEAR(c4) AS ano, letra
        FROM 
            tbl0 
            LATERAL VIEW 
                explode(c5) tbl0 AS letra
        ORDER BY
            ano, letra) t0
    GROUP BY 
        t0.ano, t0.letra
    ORDER BY 
        t0.ano, t0.letra;     
--SELECT * 
--FROM salida; 
--LIMIT 5;     

DROP TABLE IF EXISTS salida;
OK
Time taken: 0.16 seconds
CREATE TABLE salida
AS
    SELECT t0.ano, t0.letra, count(*) AS conteo
    FROM
        (
        SELECT YEAR(c4) AS ano, letra
        FROM 
            tbl0 
            LATERAL VIEW 
                explode(c5) tbl0 AS letra
        ORDER BY
            ano, letra) t0
    GROUP BY 
        t0.ano, t0.letra
    ORDER BY 
        t0.ano, t0.letra;     
Query ID = root_20200112200724_d2d9531d-d72e-4f07-ac92-d40441c9209b
Total jobs = 2
Launching Job 1 out of 2
Number of reduce tasks determined at compile time: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1578847302203_0019, Tracking URL = http://ae3ea49e5722:8088/proxy/application_1578847302203_0019/
Kill Command = /usr/loc

In [10]:
%%hive
INSERT OVERWRITE DIRECTORY 'tmp/000000_0'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT
    *
FROM
    salida;

INSERT OVERWRITE DIRECTORY 'tmp/000000_0'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT
    *
FROM
    salida;
Query ID = root_20200112200844_3f6374fb-4c5d-4f41-934b-1159e5592f4e
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1578847302203_0021, Tracking URL = http://ae3ea49e5722:8088/proxy/application_1578847302203_0021/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1578847302203_0021
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2020-01-12 20:08:56,826 Stage-1 map = 0%,  reduce = 0%
2020-01-12 20:09:05,028 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 2.13 sec
MapReduce Total cumulative CPU time: 2 seconds 130 msec
Ended Job = job_1578847302203_0021
Stage-3 is selected by condition resolver.
Stage-2 is filtered out by condition resolver.
Stage-4 is filtered out by condition resolver.
Moving data to directory hdfs://0.0.0.0:9000/user/root/tmp/000000_

In [11]:
!hadoop fs -copyToLocal ./tmp/000000_0 output

In [12]:
%hive_quit